In [ ]:
"""Basic word2vec example."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
import math
import os
import random
import sys
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.contrib.tensorboard.plugins import projector

data_index = 0

def word2vec_basic(log_dir):
  def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
      data = tf.compat.as_str(f.read(f.namelist()[0])).split()
      print(data)
    return data
#runable
  #filename="merged_wiki_text_with_only_utf8_characters_cut_subsample1.zip"
  filename="merged_wiki_text_with_only_utf8_characters_cut_subsample_subsample.zip"
    
  vocabulary = read_data(filename)
  #Nearest toprint(vocabulary)
  print("hi")
  #print(vocabulary[0])
  #print('length of voca', len(vocabulary))
# Step 2: Build the dictionary and replace rare words with UNK token.
  #i think this size close to the true word size is better
  #ACtual size is about 1826528
  vocabulary_size = 1600000

  def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    print(count[2])
    dictionary = dict()
    #這邊用dictionary length 把所有單字編號
    #
    for word, _ in count:
      dictionary[word] = len(dictionary)
      #print(len(dictionary))
    data = list()
    unk_count = 0
    for word in words:
      index = dictionary.get(word, 0)
    ###########################################################################
    #this is the way to that those word which is not included to labeled zero #
    #instead of using dictionary["word"] to get will cuuse error              #
    ###########################################################################
      if index == 0: # dictionary['UNK']
        unk_count += 1
      data.append(index)
    #把unk的數目
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    print("type=",type(dictionary.values()))
    #print(dictionary["shithole"])
    print(dictionary.get("culottes",0))
    return data, count, dictionary, reversed_dictionary

  # Filling 4 global variables:
  # data - list of codes (integers from 0 to vocabulary_size-1).
  #   This is the original text but words are replaced by their codes
  # count - map of words(strings) to count of occurrences
  # dictionary - map of words(strings) to their codes(integers)
  # reverse_dictionary - maps codes(integers) to words(strings)
  data, count, unused_dictionary, reverse_dictionary = build_dataset(vocabulary, vocabulary_size)
  print('dict_size=',len(reverse_dictionary))
  del vocabulary  # Hint to reduce memory.
  #rint('Most common words (+UNK)', count[:5])
 #print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])
  #rint('counter=', coun
  
  # Step 3: Function to generate a training batch for the skip-gram model.
  #batch_size=> 一次load 幾個data number_skip=> 包含target+2*window_size skip_window=> 左右的window_size
  def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    #建立斷句，讓
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    #創建一個deque，可以從左右端加入element的list,maxlen 規定了這個list 的最大長度，
    #若在達到最大長度時新增物件進去，則會從另外一端跳出相同數量的物件
    buffer = collections.deque(maxlen=span)  # pylint: disable=redefined-builtin
    #rint("collection.deque=",collections.deque(maxlen=span))
    if data_index + span > len(data):
      data_index = 0
    #extend(iterable)，扩展deque的右侧，通过添加iterable参数中的元素。
    buffer.extend(data[data_index:data_index + span])
    #data[x:x+k] 出來data[x,x+1,...,x+k-1]
    #print("buffer=",data[data_index:data_index])
    #更新data_index
    data_index += span
    for i in range(batch_size // num_skips):#整除 取除數
      context_words = [w for w in range(span) if w != skip_window]#w=skip window時，w為target
      #rint("context_words=",context_words)
      words_to_use = random.sample(context_words, num_skips)
      #rint("word_to_use=",words_to_use)
      for j, context_word in enumerate(words_to_use):
        batch[i * num_skips + j] = buffer[skip_window]
        labels[i * num_skips + j, 0] = buffer[context_word]
      if data_index == len(data):
        buffer.extend(data[0:span])
        data_index = span
      else:
        buffer.append(data[data_index])
        data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels
  #這邊設定一些參數，使用到的單字數量為batch_size_num/num_skips_num 每個單字會對應出 skips_num的輸出
  batch_size_num=40;
  num_skips_num=10
  skip_window_num=5
  batch, labels = generate_batch(batch_size=batch_size_num, num_skips=num_skips_num, skip_window=skip_window_num)
  print(batch)
  print(labels)
  print("dict_size=",len(reverse_dictionary))
  for i in range(batch_size_num):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0],
          reverse_dictionary[labels[i, 0]])
    
  
  ######################################step 4#########################################
  # Step 4: Build and train a skip-gram model.
  batch_size = 100
  embedding_size = 300#imension of the embedding vector.
  skip_window = 10  # How many words to consider left and right.
  num_skips = 10  # How many times to reuse an input to generate a label.
  num_sampled = 10  # Number of negative examples to sample.

  # We pick a random validation set to sample nearest neighbors. Here we limit
  # the validation samples to the words that have a low numeric ID, which by
  # construction are also the most frequent. These 3 variables are used only for
  # displaying model accuracy, they don't affect calculation.
  valid_size = 30  # Random set of words to evaluate similarity on.
  valid_window = 100  # Only pick dev samples in the head of the distribution.
  valid_examples = np.random.choice(valid_window, valid_size, replace=False)

  graph = tf.Graph()
  
  with graph.as_default():
  #根據graph 的結構來分層，層級(好像)不會影響到執行的順序跟結果，但是graph的樣貌會長得很整齊
    # Input data.
    with tf.name_scope('inputs'):
      train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
      train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
      valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
      # Look up embeddings for inputs.
      #這邊把input對應成向量
      with tf.name_scope('embeddings'):
        #with tf.Session() as sess:
            #tf.global_variables_initializer()
            #print("train_inputs=",sess.run(embeddings))
        #print("train_inputs=",train_inputs.eval())
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        #train_input 裡面會長得像這樣:[5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156] 然後lookup就會去一個一個把，embeddings裡面的第5234
        #的向量給拉出來
      # Construct the variables for the NCE loss
      #這邊把parameter的向量construct 出來
      with tf.name_scope('weights'):
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],stddev=1.0 / math.sqrt(embedding_size)))
      with tf.name_scope('biases'):
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    # Explanation of the meaning of NCE loss:
    #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
    with tf.name_scope('loss'):
      loss = tf.reduce_mean(
          tf.nn.nce_loss(
              weights=nce_weights,
              biases=nce_biases,
              labels=train_labels,
              inputs=embed,
              num_sampled=num_sampled,
              num_classes=vocabulary_size))

    # Add the loss value as a scalar to summary.
    tf.summary.scalar('loss', loss)

    # Construct the SGD optimizer using a learning rate of 1.0.
    with tf.name_scope('optimizer'):
      optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all
    # embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,
                                              valid_dataset)
    #####################################################################################################################
    #Why there's a similarty in here while no in nce loss? I change here and observe the result                         #
    #####################################################################################################################
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Merge all summaries.
    merged = tf.summary.merge_all()

    # Add variable initializer.
    init = tf.global_variables_initializer()

    # Create a saver.
    saver = tf.train.Saver()
    ##############################################################################################################################
     # Step 5: Begin training.
  num_steps = 5000001
#這邊決定了一個graph去做
  with tf.Session(graph=graph) as session:
    # Open a writer to write summaries.
    writer = tf.summary.FileWriter(log_dir, session.graph)

    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_steps):
      batch_inputs, batch_labels = generate_batch(batch_size, num_skips,
                                                  skip_window)
      feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

      # Define metadata variable.
      run_metadata = tf.RunMetadata()

      # We perform one update step by evaluating the optimizer op (including it
      # in the list of returned values for session.run()
      # Also, evaluate the merged op to get all summaries from the returned
      # "summary" variable. Feed metadata variable to session for visualizing
      # the graph in TensorBoard.
      _, summary, loss_val = session.run([optimizer, merged, loss],
                                         feed_dict=feed_dict,
                                         run_metadata=run_metadata)
      average_loss += loss_val

      # Add returned summaries to writer in each step.
      writer.add_summary(summary, step)
      # Add metadata to visualize the graph for the last run.
      if step == (num_steps - 1):
        writer.add_run_metadata(run_metadata, 'step%d' % step)

      if step % 2000 == 0:
        if step > 0:
          average_loss /= 2000
        # The average loss is an estimate of the loss over the last 2000
        # batches.
        print('Average loss at step ', step, ': ', average_loss)
        average_loss = 0

      # Note that this is expensive (~20% slowdown if computed every 500 steps)
      if step % 10000 == 0:
        sim = similarity.eval()
        for i in xrange(valid_size):
          valid_word = reverse_dictionary[valid_examples[i]]
          top_k = 8  # number of nearest neighbors
          nearest = (-sim[i, :]).argsort()[1:top_k + 1]
          log_str = 'Nearest to %s:' % valid_word
          for k in xrange(top_k):
            close_word = reverse_dictionary[nearest[k]]
            log_str = '%s %s,' % (log_str, close_word)
          print(log_str)
    final_embeddings = normalized_embeddings.eval()

    # Write corresponding labels for the embeddings.
    with open(log_dir + '/metadata.tsv', 'w') as f:
      lk=0
      for i in xrange(vocabulary_size):
        try:
            f.write(reverse_dictionary[i] + '\n')
        except:
            lk=1  

    # Save the model for checkpoints.
    saver.save(session, os.path.join(log_dir, 'model.ckpt'))

    # Create a configuration for visualizing embeddings with the labels in
    # TensorBoard.
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = embeddings.name
    embedding_conf.metadata_path = os.path.join(log_dir, 'metadata.tsv')
    projector.visualize_embeddings(writer, config)

  writer.close()

  # Step 6: Visualize the embeddings.

  # pylint: disable=missing-docstring
  # Function to draw visualization of distance between embeddings.
  def plot_with_labels(low_dim_embs, labels, filename):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
      x, y = low_dim_embs[i, :]
      plt.scatter(x, y)
      plt.annotate(
          label,
          xy=(x, y),
          xytext=(5, 2),
          textcoords='offset points',
          ha='right',
          va='bottom')

    plt.savefig(filename)

  try:
    # pylint: disable=g-import-not-at-top
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(
        perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels, os.path.join(gettempdir(),
                                                        'tsne.png'))

  except ImportError as ex:
    print('Please install sklearn, matplotlib, and scipy to show embeddings.')
    print(ex)


# All functionality is run after tf.app.run() (b/122547914). This could be split
# up but the methods are laid sequentially with their usage for clarity.
def main(unused_argv):
  # Give a folder path as an argument with '--log_dir' to save
  # TensorBoard summaries. Default is a log folder in current directory.
  current_path = os.path.dirname(os.path.realpath(sys.argv[0]))

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--log_dir',
      type=str,
      default=os.path.join(current_path, 'log'),
      help='The log directory for TensorBoard summaries.')
  flags, unused_flags = parser.parse_known_args()
  word2vec_basic(flags.log_dir)
word2vec_basic("logdir")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



('在', 13582)
type= <class 'dict_values'>
0
dict_size= 1600000
[ 22790  22790  22790  22790  22790  22790  22790  22790  22790  22790
  11168  11168  11168  11168  11168  11168  11168  11168  11168  11168
  38148  38148  38148  38148  38148  38148  38148  38148  38148  38148
 458852 458852 458852 458852 458852 458852 458852 458852 458852 458852]
[[  1507]
 [     7]
 [ 38148]
 [155993]
 [   125]
 [   117]
 [   902]
 [458852]
 [ 11168]
 [  3507]
 [   902]
 [ 22790]
 [458852]
 [   125]
 [155993]
 [  3507]
 [   117]
 [ 38148]
 [ 50852]
 [  1507]
 [  1507]
 [ 11168]
 [   902]
 [155993]
 [   125]
 [573440]
 [458852]
 [ 50852]
 [ 22790]
 [   117]
 [   902]
 [ 50852]
 [   117]
 [   231]
 [  1507]
 [ 22790]
 [ 11168]
 [ 38148]
 [155993]
 [573440]]
dict_size= 1600000
22790 如果說 -> 1507 不足
22790 如果說 -> 7 個
22790 如果說 -> 38148 克萊恩
22790 如果說 -> 155993 乾渴
22790 如果說 -> 125 說
22790 如果說 -> 117 位
22790 如果說 -> 902 投資
22790 如果說 -> 458852 焦渴
22790 如果說 -> 11168 勞倫斯
22790 如果說 -> 3507 不如
11168 勞倫斯 -> 902 投資
1116

Nearest to 會: 山麓, 電臺, 制, 並存, 烏桓, 羅馬人, 樂山, 建,
Nearest to 到: 哪, 設定, 干涉, 討, 古代, 僅有, 促使, 成功,
Average loss at step  32000 :  42.660560230851175
Average loss at step  34000 :  41.933865897715094
Average loss at step  36000 :  42.489675372838974
Average loss at step  38000 :  41.71554252481461
Average loss at step  40000 :  40.46661042225361
Nearest to 年: 眾, 主流, 單位, 佔領, 大宋, 面積, 執照, 黃河,
Nearest to 時: 元代, 宦官, 博士, 進步, 汗, 名勝區, 詮釋, 合同,
Nearest to 從: 劃歸, 行業, 師專, 出兵, 如是, 墓, 四月, 車站,
Nearest to 被: 唱, 零售額, 吃, 顯德, 障礙, 黃仁宇, 突出, 優先,
Nearest to 則: 營運, 家鄉, 減少, 閩南人, 中原, 舒穆魯氏, 社會學, 憲法,
Nearest to 月: 加工, 中等, 此外, 資格, 創造出, 協商, 全, 為主,
Nearest to 多: 音, 號稱, 電請, 開元, 予, 毫米, 世, 聽,
Nearest to 也: 保障, 鄭, 首府, 基督, 貝爾, 著稱, 腳踏車, 表明,
Nearest to 由: 上海市, 法院, 【, 據, 加以, 對應, 可, 列表,
Nearest to 站: 定向, 長江, 普魯塔克, 書店, 州郡, 東, 文, 到達,
Nearest to 和: 秦情, 加工, 台灣, 鎮壓, 羅耀雄, 火星, 設立, 巴努古萊,
Nearest to 不: 歲, 原住民, 蒙古族, 舞, 述, 信息, 組成, 援引,
Nearest to 而: 做, 相互, 鎮壓, 遷至, 立, 安西, 散文, 告密,
Nearest to 用: 珠江, 會師, 影視城, 刺史, 核心, 從事, 正式, 結算,
Nearest to 他: 泰雅族, 普林斯

Nearest to 到: 設定, 成功, 綠營, 菲律賓海, 歷經, 流徒, 哪, 單複數,
Average loss at step  82000 :  34.05278191554547
Average loss at step  84000 :  33.04346227312088
Average loss at step  86000 :  33.07313684749603
Average loss at step  88000 :  31.976403856128453
Average loss at step  90000 :  33.09161663430929
Nearest to 年: 眾, 對於, 預, 複合物, 美利堅, 通信, 另外, 疑案,
Nearest to 時: 東岸, 宦官, 聊天室, 勢力, 二二八, 依然, 詮釋, 內人黨,
Nearest to 從: 如是, 從而, 持續, 同事, 進行, 車站, 視, 劃歸,
Nearest to 被: 優勢, 唐人街, 違, 唱, 宋江, 貨幣, 王金平, 吃,
Nearest to 則: 年輕人, 戲曲, 市中心, 喜歡, 閩南人, 營運, 準, 畢,
Nearest to 月: 談, 文章, ᠨ, 評價, 成, 雲山, 認同, 下官,
Nearest to 多: 音, 貝辛斯托克, 虛擬機, 滇池, 場地, 設定, 中國, 年正式,
Nearest to 也: 保障, 偏旁, は, 學者, 政策, 申, 王位, 激活,
Nearest to 由: 論語, 驚恐, 製造商, 多媒體, 義和團, 其它, 列表, 亨德,
Nearest to 站: 醫學, 書店, 耐, 初, 民意, 文, 貨幣, 立,
Nearest to 和: 案件, 水運, 趙爾巽, 鎮海三, 共同體, 名城, 肅清, 委員會,
Nearest to 不: 總站, 信息, 南洋, 組成, 國內, 掌, 火槍, 偽書,
Nearest to 而: 以及, 相互, 遺囑, 棉蘭老島, 一直, 惟有, 回紇, 稍稍,
Nearest to 用: 會師, 所有, 版本, 珠江, 處, 正式, 影視城, 主義,
Nearest to 他: 封地, 壓力, 飛, 授權, 在, 鎂, 之後, 語句,
Nearest to 此: 

Nearest to 到: 成功, 歷經, 同意, 發起, 常任, 領土, 菲律賓海, 能夠,
Average loss at step  132000 :  30.210002736330033
Average loss at step  134000 :  29.375895979702474
Average loss at step  136000 :  29.454675285458563
Average loss at step  138000 :  30.46472954750061
Average loss at step  140000 :  29.966728386342524
Nearest to 年: 投進, 新月, 預, 銀行, 成果獎, 越來越, 建成, 樹立,
Nearest to 時: 使用, 宦官, 聊天室, 博士, 內人黨, 東岸, 討厭, 致死,
Nearest to 從: 同事, 進行, 如是, 恐慌, 無, 綠色, 西突厥, 四川省,
Nearest to 被: 儒家, 違, 著作權, 格物, 交換律, 分別, 優勢, 胤禵,
Nearest to 則: 孫子, 溫家寶, 市中心, 升起, 文, 足協, 聽說, 準,
Nearest to 月: 雲山, 族, 評價, 客觀, 矩, 不予, 足, 史瓦辛格,
Nearest to 多: 虛擬機, 倉, 反應, 音, 程度, 不過, 吸取, 號稱,
Nearest to 也: 喬納森, 保障, 來自, 李遠, 變化, 門檻, 脊椎, 新增,
Nearest to 由: 北美, 影展, 多媒體, 僅, 列表, 解釋, 佈置, 驚恐,
Nearest to 站: 醫學, 魏徵, 耐, 到達, 帶動, 積, 工程師, 陪都,
Nearest to 和: 那麼多, 遵義, 肅清, 按照, 利維, 黃華, 師友, 隸屬於,
Nearest to 不: 傷亡, 穎, 組成, 總站, 信息, 非, 火槍, 司法,
Nearest to 而: 近代, 九夷, 羅氏, 甚至, 像, 以及, 反彈, 告密,
Nearest to 用: 所有, 欲, 一般, 單字, 影視城, 如何, 編寫, 遺址,
Nearest to 他: 一切, 論, 唃廝, 手持, 起事, 北伐軍, 作風, 開設,
Neares

...

Nearest to 或: 定義, 無法, 正確, 以來, 施行, 對於, 例子, 詳細,
Nearest to 國家: 同大, 地, 統一, 眾多, 任職, 今日, 獲得, 賓館,
Nearest to 又: 承, 城, 自身, 制度, 對於, 不過, 投降, 規定,
Nearest to 了: 建立, 生活, 這, 政黨, 駐, 意為, 繼承, 豐富,
Nearest to 名: 作為, 用, 普通, 拉托, 餘現, 身份, 百, 製造,
Nearest to 會: 布丹, 給, 象徵, 意, 朋友, 下, 配基, 強烈,
Nearest to 到: 繼續, 一審, 兼, 方式, 列為, 資料, 中國, 傅樂成,
Average loss at step  442000 :  19.55948098909855
Average loss at step  444000 :  20.250386061310767
Average loss at step  446000 :  19.381599487379194
Average loss at step  448000 :  19.81114994698763
Average loss at step  450000 :  18.648309447050096
Nearest to 年: ẕ, 設立, 世界, 美, 以前, 名為, 競選, 把,
Nearest to 時: 久, 原先, 頗, 開科, 註意, 說, 親自, 根據,
Nearest to 從: 世紀, 如何, 合肥, 排除, 人類, 中華, 布丹, 游行,
Nearest to 被: 設置, 成立, 本土, 拒絕, 制定, 甘露, 韓, 經常,
Nearest to 則: 世紀, 結果, 大規模, 基礎, 符合, 布丹, 它們, 是,
Nearest to 月: 蘇聯, 推動, 失去, 大屠殺, 分別, 以後, 布丹, 國會,
Nearest to 多: 亦, 日軍, 專輯, 以前, 少, 地位, 戰略, 軍艦,
Nearest to 也: 像, 因為, 下來, 女兒, 之, 它們, 理, 華人,
Nearest to 由: 優勢, 逐漸, 類似, 以及, 或者, 相同, 中文, 這些,
Nearest to 站: 九品, 列表, 魏徵, 因, 利維

Average loss at step  492000 :  18.941270910471676
Average loss at step  494000 :  19.1066763278842
Average loss at step  496000 :  17.809685682088137
Average loss at step  498000 :  18.475853426635265
Average loss at step  500000 :  18.818045951008795
Nearest to 年: 開始, 軍隊, 朝鮮, 人民, 設立, 戰, 些, 兄弟,
Nearest to 時: 質疑, 因而, 久, 數十, 然後, 山, 開科, 深,
Nearest to 從: 形成, 內部, 從此, 本身, 居, 說, 以來, 燕,
Nearest to 被: 配基, 布丹, 作為, 誕生, 相同, 針對, 治理, 協會,
Nearest to 則: 十五, 結果, 必要, 有, 永久, 未, 相應, 之下,
Nearest to 月: 車輛, 里, 十六, 航展, 僅有, 命令, 下令, 經常,
Nearest to 多: 取, 軍艦, 並未, 一路, 場合, 國, 足以, 福建,
Nearest to 也: 價值, 多數, 業, 事情, 之, 甚至, 下來, 開始,
Nearest to 由: 組成, 布丹, 具體, 南海府, 採取, 開始, 擴張, 針對,
Nearest to 站: 連接, 第三十一, 經營, 福建, 齊, 九品, 深圳, 覆蓋,
Nearest to 和: 導致, 布丹, 又, 產生, 重大, 理想, 穩定, 直至,
Nearest to 不: 不可, 大, 表明, 創造, 應該, 經歷, 行為, 中國人,
Nearest to 而: 早期, 對抗, 當中, 汝, 布丹, 起來, 從, 選擇,
Nearest to 用: 角動量, 格, 數, 協作, 長達, 短, 後代, 位置,
Nearest to 他: 同, 起來, 乃, 之, 表示, 相關, 聽, 發表,
Nearest to 此: 死亡, 利益, 表明, 於是, 各, 證據, 布丹, 提及,
Nearest to 對: 幾乎, 然而, 威脅, 敵, 無, 秘密

Nearest to 時: 擁有, 海曼, 兒子, 管理, 之中, 其後, 變成, 學習,
Nearest to 從: 許多, 重要, 變, 完整, 一般, 設置, 形態, 可能,
Nearest to 被: 另外, 之後, 證據, 相對論, 協會, 檢查, 堅持, 國民軍,
Nearest to 則: 事件, 秩序, 一般, 市場, 約, 強調, 之後, 二百,
Nearest to 月: 取得, 中心, 樞紐, 到, 總部, 廣州, 媒體, 協會,
Nearest to 多: 美軍, 適合, 之外, 場合, 幾, 即, 足以, ẕ,
Nearest to 也: 融合, 當中, 證據, 那些, 標準, 一切, 繼承, 之前,
Nearest to 由: 化, 和, 貿易, 以, 特區, 南海府, 隨著, 布丹,
Nearest to 站: 深圳, 連接, 美國, 需, 廣州, 公立, 長沙, 路線,
Nearest to 和: 全面, 由, 轄省, 交通, 水平, 等等, 對峙, 其後,
Nearest to 不: 不可, 原則, 本身, 有所, 英, 者, 比較, 所,
Nearest to 而: 效果, 創造, 家, 不, 當中, 公司, 特征, 手,
Nearest to 用: 併, 因, 之, 無論, 方式, 那麼, 津秦, 將,
Nearest to 他: 同, 以, 發達, 經常, 突破, 止, 針對, 寧,
Nearest to 此: 既, 進一步, 即, 不斷, 考, 布丹, 首領, 得,
Nearest to 對: 件, 容易, 戰勝, 海曼, 追求, 效果, 要, 關心,
Nearest to 香港: 為主, 十五, 股份, 美國, 關於, 聽, 不要, 第四,
Nearest to 向: 中華民國, 目標, 議題, 不再, 不久, 道路, 條件, 加以,
Nearest to 是: 以前, 章, 比較, 單位, 星體, 結果, 皮膚, 稱為,
Nearest to 兩: 最初, 之內, 一直, 布丹, 供, 而, 不, 針對,
Nearest to 於: 鐵路, 投資, 後來, 並且, 作家, 當中, 知名, 極,
Nearest to 全: 布丹, 被迫, 評論, 運動, 位置, 證據, 這時, 譜號,
Nearest to 或: 自然, 

Nearest to 由: 及, 布丹, 這, 國民黨, 朋友, 提高, 以, 這裡,
Nearest to 站: 博士, 路線, 南京, 計算機, 無錫, 側, 臺北市, 同大,
Nearest to 和: 物, 國內, 治, 無法, 美, 起, 尤利西, 明,
Nearest to 不: 使得, 解決, 論述, 既, 接受, 內容, 布丹, 很少,
Nearest to 而: 混亂, 起來, 留下, 情況, 常, 通過, 在一起, 這,
Nearest to 用: 所謂, 步, 空間, 計算, 正, 黃金, 這裡, 參考,
Nearest to 他: 想, 父親, 其實, 現任, 撰寫, 獨立, 詹姆斯, 日,
Nearest to 此: 既, 處於, 從此, 考, 評價, 繼續, 隨著, 立,
Nearest to 對: 追求, 無論, 威脅, 難以, 行為, 正, 壓力, 意圖,
Nearest to 香港: 知名, 總部, 子公司, 張國, 旅遊, 女, 樂壇, 無錫,
Nearest to 向: 才, 上升, 派出, 布丹, 部隊, 隨著, 之間, 利用,
Nearest to 是: 關鍵, 基本, 用來, 融合, 沉迷, 稱為, 否定, 提,
Nearest to 兩: 選舉, 之內, 長春, 早, 對抗, 年底, 一直, 地位,
Nearest to 於: 最後, 從, 北美, 之, 三十, 國內, 並且, 引入,
Nearest to 全: 方面, 布丹, 第二, 君琰, 廢除, 堅持, 指揮, 分,
Nearest to 或: 不會, 能夠, 試圖, 大多數, 發言, 拉丁文, 子王, 如果,
Nearest to 國家: 學會, 關於, 回應, 計劃, 當代, 參見, 碩士, 外國,
Nearest to 又: 背景, 布丹, 整, 變得, 想, 長春, 宣稱, 母,
Nearest to 了: 議, 還, 相信, 任務, 學術, 無論, 單位, 困境,
Nearest to 名: 地文, 當局, 不過, 學生, 同意, 而, 伯爵, 高雄,
Nearest to 會: 可能, 有效, 教學部, 之前, 布丹, 如果, 使得, 做為,
Nearest to 到: 頻繁, 這裡, 報道, 考察, 另, 處於, 北京, 古,
Average los

Nearest to 此: 文, 得, 半, 定, 介紹, 從此, 雖, 明,
Nearest to 對: 例子, 瞭解, 持, 會社法, 認為, 狀況, 認定, 板門店,
Nearest to 香港: 文革, 開放, 張學友, 有異, 以來, 女, 基金會, 回歸,
Nearest to 向: 會社法, 結束, 布丹, 知道, 瓦弄, 稅處, 永, 隨著,
Nearest to 是: 者, 稱為, 否定, 靈, 最早, 熟悉, 指出, 布丹,
Nearest to 兩: 品種, 黃色, 顏色, 不少, 單位, 境, 整, 保存,
Nearest to 於: 國內, 十六, 秘密, 周, 福麟, 北京, 正式, 當中,
Nearest to 全: 布丹, 國, 第二, 第一, 唯一, 總人口, 資源局, 文,
Nearest to 或: 能夠, 瞭解, 試圖, 所, 都, 聲稱, 威治文, 發明,
Nearest to 國家: 概念, 斯, 以, 項, 之中, 同意, 大衛, 先驅,
Nearest to 又: 實, 整, 勝利, 九, 海外, 所在, 應, 阿瓦利斯,
Nearest to 了: 見到, 耶路撒冷, 要, 前往, 抗議, 受到, 大為, 相信,
Nearest to 名: 殺死, 數, 共和黨, 傳統, 創造, 頭銜, 中間, 莫,
Nearest to 會: 同時, 時, 指出, 而且, 之後, 覆蓋, 之前, 交流,
Nearest to 到: 正式, 現在, 考察, 會社法, 處於, 導彈, 成姬, 前往,
Average loss at step  652000 :  16.1843517042771
Average loss at step  654000 :  16.38811754527688
Average loss at step  656000 :  17.00898837977648
Average loss at step  658000 :  15.693162214159965
Average loss at step  660000 :  16.8031541454792
Nearest to 年: 錶面, 了, 階段, 有關, 任職, 稅處, 高層, 援助,
Nearest to 時: 亦, 會, 選舉, 直接, 中